<a href="https://colab.research.google.com/github/rhansen620/StatNLP2022/blob/main/Binary_Classification_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import tensorflow as tf
import tensorflow_hub as hub
!pip install tensorflow-text
import tensorflow_text as text
from tensorflow import keras

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
!pip install pyyaml h5py


This is how I turned my .txt file to a compatable .csv file

copy text from .txt into an excel document
select the cells to split (ex. column A)
in Data tab, select texts --> columns
    select 'delimited'
    select what to separate it on (I added the @ symbol after all of my labels and I delimited on @)
    select where you want it to be (shouldn't have to change anything - click finish)
I applied =TRIM(B1) in the next column (C)
I copied and pasted as numbers and deleted columns B and C
Save as .csv

Mount to Drive

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Copy over data

In [4]:
!cp -r gdrive/MyDrive/Stat\ NLP /content/

In [5]:
!ls

 gdrive   sample_data  'Stat NLP'


In [6]:
df= pd.read_csv("Stat NLP/randomboth.csv", encoding='unicode_escape') #make sure the first row of your csv file has your column labels
df.head()
#print (df)

,Label,Text
0,FRENCH,"Et ce qui l'inquiétait surtout, devant cette p..."
1,FRENCH,"Le soir, la première table était à cinq heures."
2,ENGLISH,"On the first floor, however, a small window ov..."
3,FRENCH,"Secoue-toi un peu, ne reste pas écrasé ainsi..."
4,FRENCH,Comme il donnait pourtant les dernières pièces...


In [7]:
df['English']=df['Label'].apply(lambda x: 1 if x=='ENGLISH' else 0)
df.head()

,Label,Text,English
0,FRENCH,"Et ce qui l'inquiétait surtout, devant cette p...",0
1,FRENCH,"Le soir, la première table était à cinq heures.",0
2,ENGLISH,"On the first floor, however, a small window ov...",1
3,FRENCH,"Secoue-toi un peu, ne reste pas écrasé ainsi...",0
4,FRENCH,Comme il donnait pourtant les dernières pièces...,0


split your data

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df['Text'],df['English'], stratify=df['English'])
X_train.head(4)

6958     Elle avait conquis une autorité absolue, par s...
13008               Fortunately Baudu was not at his door.
8906     En voilà une, au moins, qui mettait le pied su...
4504     Sur le trottoir, Baudu n'adressa pas la parole...
Name: Text, dtype: object

In [91]:
#print (type(X_train))
#print (type(y_train))
#print (type(y_test))
#print (type(X_train))
#dd = list(zone[0]) #Where zone[0] is some specific column of the table
#idx = dd.index(filename[i])
#print(X_test.to_numpy())

AttributeError: ignored

BERT!

In [9]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

Bert Layers

In [99]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.string, name='text'), name='text', description="created by layer 'text'")


Neural Network Layers

In [11]:
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

Use inputs and outputs to construct a final model

In [12]:
model = tf.keras.Model(inputs=[text_input], outputs = [l])

loss

In [13]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

Train, baby, Train

In [14]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=2, batch_size = 32, callbacks=[cp_callback])

Epoch 1/2
339/339 [==============================] - ETA: 0s - loss: 0.1984 - accuracy: 0.9526 
Epoch 1: saving model to training_1/cp.ckpt
339/339 [==============================] - 4322s 13s/step - loss: 0.1984 - accuracy: 0.9526
Epoch 2/2
339/339 [==============================] - ETA: 0s - loss: 0.0533 - accuracy: 0.9918 
Epoch 2: saving model to training_1/cp.ckpt
339/339 [==============================] - 4346s 13s/step - loss: 0.0533 - accuracy: 0.9918


collection of checkpoints

In [15]:
os.listdir(checkpoint_dir)

['checkpoint', 'cp.ckpt.index', 'cp.ckpt.data-00000-of-00001']

In [16]:
!mkdir -p saved_model
model.save('saved_model/my_model') 

INFO:tensorflow:Assets written to: saved_model/my_model/assets


INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [17]:
model.save('my_model.h5') 

In [19]:
cp -r training_1 gdrive/MyDrive/Stat\ NLP

In [20]:
cp -r saved_model gdrive/MyDrive/Stat\ NLP

In [21]:
cp -r my_model.h5 gdrive/MyDrive/Stat\ NLP

In [22]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()
print(y_predicted)

[0.03006887 0.0203765  0.02419171 ... 0.04213533 0.95820165 0.9824408 ]


In [104]:
#decoded = tokenizer.decode(y_predicted)

NameError: ignored

Confusion Matrix - have not gotten this to work yet

In [102]:
print (y_predicted[90])

0.01522693


In [97]:
#X_test = X_test.to_numpy()
#X_train = X_train.to_numpy()
#y_test = y_test.to_numpy()
#y_train = y_train.to_numpy()
print(y_test)

[0 0 0 ... 0 1 1]


In [43]:
listy = np.arrange()

In [58]:
print(y_predicted.shape)

(3608,)


In [60]:
as_frame=False

In [69]:
#print(y_test[0])
print(y_predicted[0])

0.030068874


In [101]:
confusion_matrix = [{"0":0,"1":0},
                    {"0":0,"1":0}]

correct_count = 0
total_count = 0
listFN = []
listFP = []
for i in range(len(y_predicted)):
    if y_predicted[i] == y_test[i]:
        correct_count += 1
    total_count += 1
    if y_predicted[i] == 0:
        if y_test[i] == 0:
            confusion_matrix[0]["0"] += 1
        elif y_test[i] == 1:
            confusion_matrix[0]["1"] += 1
            listFP.append((X_test[i], y_test[i]))
    elif y_predicted[i] == 1:
        if y_test[i] == 0:
            confusion_matrix[1]["0"] += 1
            listFN.append((X_test[i], y_test[i]))
        elif y_test[i] == 1:
            confusion_matrix[1]["1"] += 1
            

accuracy = correct_count/total_count
print(correct_count)
print("accuracy: "+str(accuracy))
print(confusion_matrix)
print("False Negative: " + str(listFN))
print("False Positive: " + str (listFP))

0
accuracy: 0.0
[{'0': 0, '1': 0}, {'0': 0, '1': 0}]
False Negative: []
False Positive: []
